In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns
import plotly.express as px
plt.rcParams['font.sans-serif'] = 'SimHei'
plt.rcParams['axes.unicode_minus'] = False    # 负号显示的解决方案


from dash import Dash, Input, Output
# from grpc import server
import plotly.express as px
# import plotly.graph_objs as go
import dash
from dash import dcc                  # 交互式组件
from dash import html                 # 代码转html
from dash.dependencies import Input, Output         # 回调
from jupyter_dash import JupyterDash

# pip3 install jupyter-server-proxy
# pip3 install jupyter-dash

### __读取数据__

In [2]:
data = pd.read_excel('./2022年售后维修在线编辑汇总(2).xlsx')

### __填补缺失值__

In [3]:
data['问题现象'].fillna('mask1', inplace=True)
print(data['问题现象'].isna().sum())

0


In [4]:
# data.dropna(axis=0, subset=['问题原因'], inplace=True)
data['问题原因'].fillna('mask2', inplace=True)
print(data['问题原因'].isna().sum())

0


In [5]:
# data.dropna(axis=0, subset=['解决方案'], inplace=True)
data['解决方案'].fillna('mask3', inplace=True)
print(data['解决方案'].isna().sum())

0


### __处理文本信息__

In [6]:
# 考虑将部分列的英文标点转换成中文标点，并去掉末尾的换行符号

# 规格型号、问题现象、问题原因、解决方案这几列需要转换
table = {ord(f):ord(t) for f,t in zip(
        u'()!?;[],，',
        u'（）！？；【】//'                 #这里顺便把，也换成了/
    )
}
def translate_E2C(col):
    for i in range(len(data)):
        if type(data.iloc[i, col]) == str:
            data.iloc[i, col] = data.iloc[i, col].strip('\n') #去掉换行
            data.iloc[i, col] = data.iloc[i, col].translate(table)


translate_E2C(4)
translate_E2C(8)
translate_E2C(9)
translate_E2C(10)

In [7]:
# 考虑将解决方案列中 UF/V 和 A/V 中的/删去

def convert_UFV():
    temp = data[data['解决方案'].str.contains('UF/') & 
                    data['解决方案'].str.contains('V')]['解决方案']
    for i in range(len(temp.index)):
        a = data.iloc[data.index==temp.index[i], 10]
        # print(list(a)[0])
        ind = list(a)[0].index('UF/')
        # print(ind)
        data.iloc[data.index==temp.index[i], 10] = list(a)[0][:ind+2] + list(a)[0][ind+3:] #用字符串切片的方式进行删除
        # print(data.iloc[data.index==temp.index[i], 10].tolist()[0])
def convert_AV():
    temp = data[data['解决方案'].str.contains('A/') & 
                    data['解决方案'].str.contains('V')]['解决方案']
    for i in range(len(temp.index)):
        a = data.iloc[data.index==temp.index[i], 10]
        # print(list(a)[0])
        ind = list(a)[0].index('A/')
        # print(ind)
        data.iloc[data.index==temp.index[i], 10] = list(a)[0][:ind+1] + list(a)[0][ind+2:] #用字符串切片的方式进行删除
        # print(data.iloc[data.index==temp.index[i], 10].tolist()[0])


convert_UFV()
convert_AV()
data[data['解决方案'].str.contains('UF/') & 
        data['解决方案'].str.contains('V')]['解决方案'].value_counts() #验证转换完毕
data[data['解决方案'].str.contains('A/') & 
        data['解决方案'].str.contains('V')]['解决方案'].value_counts() #验证转换完毕

Series([], Name: 解决方案, dtype: int64)

In [8]:
# 考虑将问题现象列的 1.2.转换为/形式，统一一下

def convert_123():
    temp = data[data['问题现象'].str.startswith('1.')]['问题现象']
    for i in range(len(temp.index)):
        a = data.iloc[data.index==temp.index[i], 8]
        # print(list(a)[0])
        if list(a)[0][:2]=='1.': #这句 if判断没有也可以
            b = list(a)[0][2:].split('.') #利用 '.'对字符串进行分割
            for j in range(len(b)):
                if j < len(b)-1: #这一步是删除经过 split分割后的多余数字
                    b[j] = b[j][:-1]
            data.iloc[data.index==temp.index[i], 8] = '/'.join(b)
            # print(data.iloc[data.index==temp.index[i], 8].tolist()[0])
convert_123()


data[data['问题现象'].str.startswith('1.')]['问题现象'].value_counts() #验证转换完毕

Series([], Name: 问题现象, dtype: int64)

In [9]:
# 考虑部分意义相同的情况，需要进行修改，统一改成相同的表述
# 因为有些数据是形如：不上电/网络连接不上
#                   不上电-网络连接不上

def handle_backslash_123(data, temp, changedstr, judgestr='', col='问题现象'):
    '''
        data: 是一个 pandas的 DataFrame
        temp: data经过 .str.contains()筛选后的结果
        judgestr: 判断字符, 例如网络连接不上的判断字符就是 '网络', 有时候不存在 '/'以及 '1.2.3.'的情况也可以选择不传入
        changedstr: 修改后的字符, 例如网络连接不上的修改后的字符就是 '网络不通'
        i: temp的下标, 因为需要遍历来按照索引进行修改
    '''
    if col != '问题现象':
        loc = data.columns.tolist().index(col)
    else:
        loc = data.columns.tolist().index('问题现象') #找到问题现象出现的列索引
    for i in range(len(temp.index)):
        a = data.iloc[data.index==temp.index[i], loc]
        if '/' in list(a)[0] or '，' in list(a)[0]: #利用 '/'对字符串进行分割
            # print(list(a)[0])
            if '/' in list(a)[0]:
                split_str = '/'
            elif '，' in list(a)[0]:
                split_str = '，'
            b = list(a)[0].split(split_str)
            for j in range(len(b)):
                if judgestr in b[j]:
                    b[j] = changedstr
                    break
            data.iloc[data.index==temp.index[i], loc] = split_str.join(b)
        else:
            data.iloc[data.index==temp.index[i], loc] = changedstr

In [10]:
# 问题现象
# 针对 '不上电'处理，统一样式
temp = data[(data['问题现象'].str.contains('不上电') | 
                data['问题现象'].str.contains('不通电')) & 
                    (~data['问题现象'].str.contains('面板不上电'))]['问题现象']
handle_backslash_123(data, temp, changedstr='不上电', judgestr='电')


# 针对 '4G不在线'处理，统一样式
temp = data[(data['问题现象'].str.contains('不')) & 
                (data['问题现象'].str.contains('线')) & 
                    (~data['问题现象'].str.contains('TS流')) & 
                        (~data['问题现象'].str.contains('平台设备'))]['问题现象']
handle_backslash_123(data, temp, changedstr='4G不上线', judgestr='线')


# 针对 '无声音输出'处理，统一样式
temp = data[data['问题现象'].str.contains('无声') & 
                ~data['问题现象'].str.contains('喊话') & 
                    ~data['问题现象'].str.contains('USB') & 
                        ~data['问题现象'].str.contains('喇叭') & 
                            ~data['问题现象'].str.contains('监听') & 
                                ~data['问题现象'].str.contains('话筒') & 
                                    ~data['问题现象'].str.contains('FM')]['问题现象']
handle_backslash_123(data, temp, changedstr='无声音输出', judgestr='声')


# 针对 '网络不通'处理，统一样式
temp = data[data['问题现象'].str.contains('网络不通') | 
                data['问题现象'].str.contains('无网络') | 
                    data['问题现象'].str.contains('网络连不上')]['问题现象']
handle_backslash_123(data, temp, changedstr='网络不通', judgestr='网络')


# 针对 '声音异常'处理，统一样式
temp = data[data['问题现象'].str.contains('声音异常')]['问题现象']
handle_backslash_123(data, temp, changedstr='声音异常', judgestr='声音')


# 针对 '搜索不到终端'处理，统一样式
temp = data[data['问题现象'].str.contains('搜') & data['问题现象'].str.contains('终端')]['问题现象']
handle_backslash_123(data, temp, changedstr='搜索不到终端', judgestr='终端')


# 针对 '不出串口'处理，统一样式
temp = data[data['问题现象'].str.contains('不出串口') | 
                data['问题现象'].str.contains('不跳串口')]['问题现象']
handle_backslash_123(data, temp, changedstr='不出串口', judgestr='串口')


# 针对 '电源指示灯不亮'处理，统一样式
temp = data[data['问题现象'].str.contains('电源') & 
                data['问题现象'].str.contains('不亮')]['问题现象']
handle_backslash_123(data, temp, changedstr='电源指示灯不亮', judgestr='电源')


# 针对 '网络指示灯不亮'处理，统一样式
temp = data[data['问题现象'].str.contains('网络') & 
                data['问题现象'].str.contains('不亮')]['问题现象']
handle_backslash_123(data, temp, changedstr='网络指示灯不亮', judgestr='灯')


# 针对 '不断重启'处理，统一样式
temp = data[data['问题现象'].str.contains('重启') & 
                ~data['问题现象'].str.contains('U盘')]['问题现象']
handle_backslash_123(data, temp, changedstr='不断重启', judgestr='重启')

In [11]:
# 同上，问题原因列
# 针对 '检测无问题'处理，统一样式
temp = data[data['问题原因'].str.contains('检') &
                ~data['问题原因'].str.contains('SIM') & 
                    ~data['问题原因'].str.contains('研发')]['问题原因']
handle_backslash_123(data, temp, changedstr='检测无问题', judgestr='检', col='问题原因')


# 针对 '开关电源坏'处理，统一样式
temp = data[data['问题原因'].str.contains('开关电源') &
                ~data['问题原因'].str.contains('扯离')]['问题原因']
handle_backslash_123(data, temp, changedstr='开关电源坏', judgestr='源', col='问题原因')


# 针对 '通讯模块坏'处理，统一样式
temp = data[data['问题原因'].str.contains('通讯') & 
                ~data['问题原因'].str.contains('铜箔') & 
                    ~data['问题原因'].str.contains('U7') & 
                        ~data['问题原因'].str.contains('进水')]['问题原因']
handle_backslash_123(data, temp, changedstr='通讯模块坏', judgestr='通讯', col='问题原因')


# 针对 '功放芯片坏'处理，统一样式
temp = data[data['问题原因'].str.contains('功放芯片')]['问题原因']
handle_backslash_123(data, temp, changedstr='功放芯片坏', judgestr='功放', col='问题原因')


# 针对 '网络芯片坏'处理，统一样式
temp = data[data['问题原因'].str.contains('网络芯片') & 
                ~data['问题原因'].str.contains('8')]['问题原因']
handle_backslash_123(data, temp, changedstr='网络芯片坏', judgestr='网络', col='问题原因')


# 针对 'SIM卡槽坏（针脚氧化）'处理，统一样式
temp = data[data['问题原因'].str.contains('SIM') & 
                data['问题原因'].str.contains('氧化') & 
                    ~data['问题原因'].str.contains('指示灯')]['问题原因']
handle_backslash_123(data, temp, changedstr='SIM卡槽坏（针脚氧化）', judgestr='SIM', col='问题原因')


# 针对 'FB2电感坏'处理，统一样式
temp = data[data['问题原因'].str.contains('FB2') & 
                ~data['问题原因'].str.contains('烧')]['问题原因']
handle_backslash_123(data, temp, changedstr='FB2电感坏', judgestr='FB2', col='问题原因')

In [12]:
# 同上，解决方案列
# 针对 '升级程序'处理，统一样式
temp = data[(data['解决方案'].str.contains('升级') | 
                data['解决方案'].str.contains('更新')) & 
                    (~data['解决方案'].str.contains('底层')) & 
                        (~data['解决方案'].str.contains('写入'))]['解决方案']
handle_backslash_123(data, temp, changedstr='升级程序', judgestr='程序', col='解决方案')


# 针对 '更换网口座子RJ45'处理，统一样式
temp = data[(data['解决方案'].str.contains('RJ45'))]['解决方案']
handle_backslash_123(data, temp, changedstr='更换网口座子RJ45', judgestr='RJ45', col='解决方案')


# 针对 '更换功放芯片MP7770'处理，统一样式
temp = data[(data['解决方案'].str.contains('MP770')) | 
                (data['解决方案'].str.contains('MP7770'))]['解决方案']
handle_backslash_123(data, temp, changedstr='更换功放芯片MP7770', judgestr='MP', col='解决方案')


# 需要在下面的配件前面加上更换二字，可能还没列举完，需要时再继续往后添加
needchange_parts = ["VS1003","ET3157","DRV632","XW217203","BM107A-FHD006","BC857","AIC3254",
            "RTL8201","蓝牙模块MF01C","MI20A","CTB-RTDM-4G-E（旧）","FIR12N80FG","30欧电阻*2个",
            "XMEGA32D4","R31-10欧姆","STM32F407","2A250V保险管","DM9000","8pin","LPC1768FBD100",
            "电位器10K*3个","网络芯片8201","SIM卡槽","DB207S","SOT23-5","FB2","ATMEGA64A","IRF4905",
            "IRFZ48V","QN8035","CTB-RTDM-AP-C","AMS1117-3.3","S1003B","MP7770","0.3欧*2个","TL431AIDBZR",
            "新二维码150-015-03378-20201014-00190","100UF450V","GX3201H","电感L2","C221","220欧"," LL4148",
            "NJM2761","NCP1380-B","R28","R29","IC-ESOP-8","3.9电源转换芯片JW5121","LED（红）","LED（绿）",
            "LED1","LM3886","遥控天线","遥控小板","4G板","GX3201"]

In [13]:
data[data['解决方案'].str.contains('MP770')]['解决方案'].value_counts()
# data[data['解决方案'].str.contains('MP7770')]['解决方案'].value_counts()
# data[data['解决方案'].str.contains('按ECN更改')]['解决方案'].value_counts()
# 'hide'

Series([], Name: 解决方案, dtype: int64)

In [14]:
temp = data[(data['解决方案'].str.contains('ECN'))]['解决方案'].value_counts()
temp

更换GX3201H/升级程序/按ECN更改/天线座加绝缘垫                 113
更换GX3201/执行ECN（2111020001）/4G天线处加绝缘垫片/升级程序    107
更换SIM卡槽/升级程序/按ECN更改                            91
执行ECN（2111020001）/4G天线处加绝缘垫片/升级程序              89
更换GX3201H/升级程序/按ECN更改                          74
                                             ... 
更换开关电源HPT75D34/执行ECN（加天线绝缘垫）/升级程序               1
更换JWAL-75S34S5KTG2//执行ECN/升级程序                  1
更换R31-10欧姆/执行第二次ECN                             1
更换CTB-RTDM-AP-C/功放板按ECN更改（加TVS管）                1
更换GX3201H/执行ECN/升级程序                            1
Name: 解决方案, Length: 298, dtype: int64

In [15]:
# 处理多问题现象的情况

def split_multiple(problem, cnt):
    '''
        主要用于分离含有 '/'或者 '，'的问题现象
        注意要先忽略掉 '/升级程序', 再分离
    '''
    i = 0
    while i < len(problem):
        # 先处理含有多情况下的升级程序情况
        if '/升级程序' in problem[i] or "升级程序/" in problem[i]:
            if '/升级程序' in problem[i]:
                problem[i] = ''.join(problem[i].split('/升级程序'))
            if '升级程序/' in problem[i]:
                problem[i] = ''.join(problem[i].split('升级程序/'))
        if '，升级程序' in problem[i] or "升级程序，" in problem[i]:
            if '，升级程序' in problem[i]:
                problem[i] = ''.join(problem[i].split('，升级程序'))
            if '升级程序，' in problem[i]:
                problem[i] = ''.join(problem[i].split('升级程序，'))
        # 再处理正常情况
        if '/' in problem[i] or '，' in problem[i]:
            if '/' in problem[i]:
                temp = problem[i].split('/')
            elif '，' in problem[i]:
                temp = problem[i].split('，')
            problem += temp
            cnt += [cnt[i]]*len(temp)
            problem.pop(i)
            cnt.pop(i)
        else:
            i += 1

def sum_multiple(problem, cnt):
    '''
        有时候存在一个规格型号有多个问题现象, 那可能需要和之前单一的情况合并起来考虑
        problem: 是一个列表, 存放着需要绘制图像的文本数据
        cnt: 是一个列表, 存放着需要绘制图像的数值数据
        先根据文本长度排序, 短的文本在前面, 防止丢失不一样的解决方案
        有时候存在空字符串 '', 需要删掉
        之后去重
        注意在 needchange_parts中的配件需要添加更换二字
    '''
    temp = sorted(enumerate(problem), key=lambda x:len(x[1]), reverse=False)
    idx = [i[0] for i in temp]
    problem = [i[1] for i in temp]
    cnt = [cnt[i] for i in idx]

    # # 去掉 ''的情况
    # if '' in problem:
    #     ind = problem.index('')
    #     problem.pop(ind)
    #     cnt.pop(ind)

    # 去重
    pro_dict = {}
    for i in range(len(problem)):
        if problem[i] != '': #忽略 ''的情况
            if problem[i] in needchange_parts:
                problem[i] = '更换'+problem[i]
            flag = 0
            # # 对于含有相同文本的情况，只取最长的文本
            # for key in pro_dict.keys(): #这一步是在去重
            #     if problem[i] in key: #短的文本在长的里面
            #         pro_dict[key] += cnt[i] #长的值增加
            #         flag = 1
            #         break
            if flag == 0:
                if problem[i] not in pro_dict:
                    pro_dict[problem[i]] = cnt[i]
                else:
                    pro_dict[problem[i]] += cnt[i]
    
    # print(list(pro_dict.keys()), '\n', list(pro_dict.values()))
    # problem, cnt = list(pro_dict.keys()), list(pro_dict.values())
    return list(pro_dict.keys()), list(pro_dict.values()) #字典解包

In [16]:
# 记录 '成品' 与 '板件' 的型号
product_type = list(data['产品类型'].value_counts().index)[:-1]

model = {} #字典用于存放成品或者板件的型号
for i in product_type:
    model[i] = list(data[data['产品类型']==i]['规格型号'].value_counts().index)
# model['成品']
# model

In [17]:
app = JupyterDash('售后数据')
all_options = model

app.layout = html.Div([
    html.Div(
        [html.H4('售后服务数据')]
    ),
    html.Div(
        [dcc.Graph(id = "graph")],
        # style = dict(width = '64%', display = 'inline-block')
    ),
    html.Div(
        [html.P("产品类型:"),
        dcc.Dropdown(id = 'types',
                    options = [{'label': k, 'value': k} for k in all_options.keys()],
                    value = '成品', #初始值为成品
                    clearable = True
        ),
        html.P("规格型号:"),
        dcc.Dropdown(id = 'specifications',
                    clearable = True
        ),
        html.P("列名:"),
        dcc.Dropdown(id = 'columns',
                    options = ['问题现象', '问题原因', '解决方案'],
                    value = '问题现象', #初始值为问题现象
                    clearable = True
        )],
        style = dict(width = '40%', display = 'inline-block') #设置下拉列表的大小
    )
])

@app.callback(
    Output('specifications', 'options'),
    [Input('types', 'value')])
def set_type_options(select_type): #选择产品类型
    return [{'label': i, 'value': i} for i in all_options[select_type]] #反馈到下拉列表的显示区

@app.callback(
    Output('specifications', 'value'),
    [Input('specifications', 'options')])
def set_specification_value(available_options): #选择规格型号
    return available_options[0]['value'] #反馈到下拉列表的显示区


@app.callback(
    Output("graph", "figure"), #需要更新的输出
    [Input("types", "value"), #需要更新的输入
    Input("specifications", "value"),
    Input("columns", "value"),])
def generate_chart(types, specifications, columns):
    texts = []
    values = []
    i = len(texts)
    temp = data[data['规格型号']==specifications][columns].value_counts()

    texts.append(list(temp.index))
    values.append(list(temp))

    # 是否忽略缺失值
    def check_exist(textdata):
        ind = []
        check_str = ['mask1', 'mask2', 'mask3', '检测无问题']
        for str in check_str:
            if str in textdata:
                ind.append(textdata.index(str))
        return ind
    ind = sorted(check_exist(texts[i]), reverse=True)
    for j in ind:
        texts[i].pop(j)
        values[i].pop(j)

    split_multiple(texts[i], values[i])
    texts[i], values[i] = sum_multiple(texts[i], values[i])

    labels, numsdata = texts[i], values[i]
    df = pd.DataFrame(zip(labels, numsdata), columns=[columns, '次数'])
    fig = px.pie(
                df,
                values = '次数',
                names = columns,
                hole = 0.3,
                title = '型号 '+ specifications +' 的主要'+ columns,
    )
    fig.update_traces(textposition='inside')
    fig.update_layout(uniformtext_minsize=10, 
                        uniformtext_mode='hide',)
    return fig


if __name__ == '__main__':
    app.run_server(mode='external', width='80%', height=800)

Dash app running on http://127.0.0.1:8050/
